# 1. Finding unique characters for encoding

In [1]:
with open('homer.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("length of dataset: ", len(text))

length of dataset:  908201


In [4]:
#Looking at first 1000 characters
print(text[:1001])

The Project Gutenberg eBook of The Iliad
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The Iliad

Author: Homer

Translator: Samuel Butler

Release date: June 1, 2000 [eBook #2199]
                Most recently updated: August 16, 2022

Language: English

Credits: Jim TinsleyRevised by Richard Tonsing.


*** START OF THE PROJECT GUTENBERG EBOOK THE ILIAD ***




      THE ILIAD OF HOMER

      Rendered into English Prose for
      the use of those who cannot
      read the original


      by Samuel Butler




Contents


 BOOK I.
 BOOK II.
 BOOK III.
 BOOK IV.
 BOOK V.
 BOOK VI.
 BOO

In [5]:
# unique characters that occur in text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !#$%()*,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz—‘’“”•™﻿
88


# 2. Tokenization: Encoding and Decoding Strategy

I will be mapping characters to numbers and create functions to encode and decode

In [11]:
# Mapping
encode_map = { ch:i for i,ch in enumerate(chars) }
decode_map = { i:ch for i, ch in enumerate(chars) }

#encoder takes string and maps to list of integers
encode = lambda e: [encode_map[c] for c in e]
#decode takes list of integers and outputs a string
decode = lambda d: ''.join([decode_map[u] for u in d])

print(encode("hellooo friend"))
print(decode(encode("hellooo friend")))

[61, 58, 65, 65, 68, 68, 68, 1, 59, 71, 62, 58, 67, 57]
hellooo friend
